In [28]:
# import dependencies
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

In [4]:
merged_nb_df = pd.read_csv("merged_nb_df.csv")
pd.set_option('display.max_columns', None)
merged_nb_df.head()

,size_l,og,fg,abv,ibu,color,boilsize,boiltime,boilgravity,efficiency,mashthickness,pitchrate,primarytemp,review_taste,beer_abv,sugarscale_Plato,sugarscale_Specific Gravity,brewmethod_All Grain,brewmethod_BIAB,brewmethod_Partial Mash,brewmethod_extract
0,18.93,1.082,1.013,9.10,0.00,4.10,21.58,60.0,0.000,72.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0
1,7.57,1.076,1.014,8.10,33.51,3.68,8.71,60.0,1.066,75.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0
2,20.82,1.043,1.011,4.14,13.22,17.67,28.39,90.0,0.000,85.0,0.0,1.75,12.22,3.028652,4.4,0.0,1.0,1.0,0.0,0.0,0.0
3,22.71,1.069,1.017,6.75,58.76,40.00,29.34,90.0,1.053,70.0,0.0,0.00,0.00,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0
4,22.71,1.068,1.018,6.53,73.40,40.00,27.44,90.0,1.056,65.0,1.3,1.25,18.89,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0


In [5]:
# What percentage of beers scored 4 or above?
len(merged_nb_df[(merged_nb_df["review_taste"]>=4)]) / len(merged_nb_df) * 100

34.77543538038497

In [6]:
# Since beers with a rating of four or above is the top 34% of beers, let's go with those as "good" beers.
# We need a new category "good beer" with a 1 for good and 0 for bad
merged_nb_df["good_beer"]=0

In [7]:
# for all the "good beers", put a 1 in the good_beer column
merged_nb_df.loc[merged_nb_df["review_taste"].abs()>=4, "good_beer"] = 1

In [8]:
merged_nb_df.head()

,size_l,og,fg,abv,ibu,color,boilsize,boiltime,boilgravity,efficiency,mashthickness,pitchrate,primarytemp,review_taste,beer_abv,sugarscale_Plato,sugarscale_Specific Gravity,brewmethod_All Grain,brewmethod_BIAB,brewmethod_Partial Mash,brewmethod_extract,good_beer
0,18.93,1.082,1.013,9.10,0.00,4.10,21.58,60.0,0.000,72.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0,1
1,7.57,1.076,1.014,8.10,33.51,3.68,8.71,60.0,1.066,75.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0,1
2,20.82,1.043,1.011,4.14,13.22,17.67,28.39,90.0,0.000,85.0,0.0,1.75,12.22,3.028652,4.4,0.0,1.0,1.0,0.0,0.0,0.0,0
3,22.71,1.069,1.017,6.75,58.76,40.00,29.34,90.0,1.053,70.0,0.0,0.00,0.00,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0,1
4,22.71,1.068,1.018,6.53,73.40,40.00,27.44,90.0,1.056,65.0,1.3,1.25,18.89,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0,1


In [9]:
beer_class = merged_nb_df.drop(columns = ["review_taste"])

In [12]:
# Split our preprocessed data into our features and target arrays
y = beer_class.good_beer.values
X = beer_class.drop(columns = ["good_beer"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, stratify = y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Random Forest Classifier

In [14]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=200, random_state=5)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.613


In [15]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.69      0.74      0.37      0.71      0.52      0.28      1779
          1       0.43      0.37      0.74      0.40      0.52      0.26       949

avg / total       0.60      0.61      0.50      0.60      0.52      0.28      2728



## Easy Ensemble Classifier

In [24]:
# Train the EasyEnsembleClassifier
classifier = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
classifier.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [25]:
# Calculated the balanced accuracy score
predictions = classifier.predict(X_test)
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.6873167155425219


In [26]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,1284,495
Actual 1,358,591


In [27]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.78      0.72      0.62      0.75      0.67      0.45      1779
          1       0.54      0.62      0.72      0.58      0.67      0.45       949

avg / total       0.70      0.69      0.66      0.69      0.67      0.45      2728



## Neural Network

In [36]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 20
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [38]:

# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
256/256 [==============================] - 0s 743us/step - loss: 0.6392 - accuracy: 0.6480
Epoch 2/150
256/256 [==============================] - 0s 762us/step - loss: 0.6242 - accuracy: 0.6557
Epoch 3/150
256/256 [==============================] - 0s 758us/step - loss: 0.6201 - accuracy: 0.6611
Epoch 4/150
256/256 [==============================] - 0s 762us/step - loss: 0.6163 - accuracy: 0.6607
Epoch 5/150
256/256 [==============================] - 0s 743us/step - loss: 0.6143 - accuracy: 0.6681
Epoch 6/150
256/256 [==============================] - 0s 809us/step - loss: 0.6126 - accuracy: 0.6704
Epoch 7/150
256/256 [==============================] - 0s 757us/step - loss: 0.6117 - accuracy: 0.6694
Epoch 8/150
256/256 [==============================] - 0s 743us/step - loss: 0.6096 - accuracy: 0.6742
Epoch 9/150
256/256 [==============================] - 0s 805us/step - loss: 0.6094 - accuracy: 0.6716
Epoch 10/150
256/256 [==============================] - 0s 852us/step - l

In [39]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

86/86 - 0s - loss: 0.6787 - accuracy: 0.6422
Loss: 0.6787494421005249, Accuracy: 0.6422287225723267


# I think Easy Ensemble Classifier is the winner!!